In [1]:
import io
import os
import panel as pn
import numpy as np
import pandas as pd

pn.extension('vega', 'ace', 'jsoneditor', 'ipywidgets', sizing_mode='stretch_width')

In [2]:
from pathlib import Path

In [17]:
# PATH = '../output'
# # SUBJ  = 'sub_00'
# check_folders()


# def create_output_folder():
#     # verify folders exist
#     check_folders()

#     # patient specific folders
#     sub = os.path.join(PATH, SUBJ)
#     net = os.path.join(sub, 'net')

#     if not os.path.exists(sub):
#         print(f'Creating folder `{sub}`...')
#         os.mkdir(sub)

#         print(f'Creating folder `{net}`...')
#         os.mkdir(net)

#     else:
#         # TODO: add create new id creation
#         pass


# def check_folders():
#     eq    = os.path.join(PATH, 'eq')
#     code  = os.path.join(PATH, 'code')
#     coor  = os.path.join(PATH, 'coord')
#     param = os.path.join(PATH, 'param')

#     for p in [path, eq, code, coor, param]:
#         if not os.path.exists(p):
#             print(f'Creating folder `{os.path.basename(p)}`...')
#             os.mkdir(p)

#     read  = os.path.join(PATH, 'README.txt')
#     part  = os.path.join(PATH, 'participants.tsv')
#     desc  = os.path.join(PATH, 'dataset_description.json')
#     chgs  = os.path.join(PATH, 'CHANGES.txt')

#     for p in [read, part, desc, chgs]:
#         if not os.path.exists(p):
#             print(f'Creating file `{os.path.basename(p)}`...')
#             Path(p).touch()


# def gen_watch(event):
#     print(event)
#     create_output_folder()

# # create_output_folder()

In [18]:
# # descriptor for JSONEditor usage
# markdown = pn.pane.Markdown("""
# ## Welcome!

# Here you can select the folder(s) that you want to transform. Beware that we are using recursive walk to select all the content
# available in the specified folder. That means if the folder contains a sub-folder, we will transform the content if it falls
# into the accepted file formats.

# Below you will see the generated folder with content as specified at [BIDS Computational Model Specification](https://docs.google.com/document/d/1NT1ERdL41oz3NibIFRyVQ2iR8xH-dKY-lRCB4eyVeRo/edit?usp=sharing).
# If you are happy with the results, press `Transform Files` button at the bottom of the screen. We will not start generation
# until you press the button below.

# ### Select Folder(s)
# """
# )


# generated_md = pn.pane.Markdown("""
# ### Generated Folder

# ```
# tsv/
#     example1.tsv
#     example2.tsv
# json/
#     example1.json
#     example2.json

# ```
# """)

# # get current path
# path = os.getcwd()
# change_path = pn.widgets.FileSelector(path)

# generate_btn = pn.widgets.Button(name='Transform Files', button_type='primary')
# generate_btn.on_click(gen_watch)

# main_area = pn.Column(
#                 markdown,
#                 change_path,
#                 generated_md,
#                 generate_btn
#             )
# main_area

In [19]:
import sys
sys.path.append("..")
from incf.preprocess import preprocess as prep

IDS = {}


def check_file(fname=None, value=None):
    s_id = prep.create_uuid()

    if fname == 'weights.txt':
        val = create_layout({'name': s_id,
                       'desc': 'default',
                       'fname': fname
                      })

        IDS[s_id] = {
            'id': s_id,
            'fname': fname
        }

        create_output_folder(sub=f'sub_{s_id}')
        fname = f'sub-{s_id}_desc-default_{os.path.basename(fname)}.tsv'
        string_io = io.StringIO(value.decode("utf8"))
        pd.read_csv(string_io, sep='\t').to_csv(os.path.join('../output', f'sub_{s_id}', 'net', fname), sep='\t',
                                                header=None, index=None)
        Path(os.path.join('../output', f'sub_{s_id}', 'net', f'{fname}.json')).touch()
        return val


def create_output_folder(path='../output', sub='sub_00'):
    # verify folders exist
    check_folders(path)

    # patient specific folders
    sub = os.path.join(path, sub)
    net = os.path.join(sub, 'net')

    if not os.path.exists(sub):
        print(f'Creating folder `{sub}`...')
        os.mkdir(sub)

        print(f'Creating folder `{net}`...')
        os.mkdir(net)

    else:
        # TODO: add create new id creation
        pass


def check_folders(path):
    eq    = os.path.join(path, 'eq')
    code  = os.path.join(path, 'code')
    coor  = os.path.join(path, 'coord')
    param = os.path.join(path, 'param')

    for p in [path, eq, code, coor, param]:
        if not os.path.exists(p):
            print(f'Creating folder `{os.path.basename(p)}`...')
            os.mkdir(p)

    read  = os.path.join(path, 'README.txt')
    part  = os.path.join(path, 'participants.tsv')
    desc  = os.path.join(path, 'dataset_description.json')
    chgs  = os.path.join(path, 'CHANGES.txt')

    for p in [read, part, desc, chgs]:
        if not os.path.exists(p):
            print(f'Creating file `{os.path.basename(p)}`...')
            Path(p).touch()


# create_output_folder()
def create_layout(subs=None):
    return f"""
    output/ <br>
        &emsp;|___ code <br>
        &emsp;|___ coord <br>
        &emsp;|___ eq <br>
        &emsp;|___ param <br>
        &emsp;{create_sub(subs)}
        &emsp;|___ README <br>
        &emsp;|___ CHANGES <br>
        &emsp;|___ dataset_description.json <br>
        &emsp;|___ participants.tsv
    """

def create_sub(sub):
    if sub is not None:
        if sub['fname'] == 'weights.txt':
            return f"""
            |___ sub-{sub['name']} <br>
                     &emsp;&emsp;&emsp;|__ net <br>
                         &emsp;&emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_weights.tsv <br>
                         &emsp;&emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_weights.json <br>
                     &emsp;&emsp;&emsp;|__ spatial <br>
                     &emsp;&emsp;&emsp;|__ ts  <br>
            """

        elif sub['fname'] == 'distances.txt':
            return f"""
                &emsp;|___ sub-{sub['name']} <br>
                     &emsp;&emsp;|__ net <br>
                         &emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_distances.tsv <br>
                         &emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_distances.json <br>
                     &emsp;&emsp;|__ spatial <br>
                     &emsp;&emsp;|__ ts  <br>
            """
    return ''
#         return f"""
#             |___ sub-{sub['name']}
#                  |__ net
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_distances.tsv
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_distances.json
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_weights.tsv
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_weights.json
#                  |__ spatial
#                  |__ ts
#         """



In [20]:
IDS

{}

In [7]:
import param


class FileSelector(param.Parameterized):
    gen_btn = pn.widgets.Button(name='Transform Files', button_type='primary')
    file_in = param.Parameter()
    txt     = param.String()


    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.FileInput(), **params)
        self.static_text = pn.widgets.StaticText()

        self.markdown = pn.pane.Markdown("""
        ## Welcome!

        Here you can select the folder(s) that you want to transform. Beware that we are using recursive walk to select all the content
        available in the specified folder. That means if the folder contains a sub-folder, we will transform the content if it falls
        into the accepted file formats.

        Below you will see the generated folder with content as specified at [BIDS Computational Model Specification](https://docs.google.com/document/d/1NT1ERdL41oz3NibIFRyVQ2iR8xH-dKY-lRCB4eyVeRo/edit?usp=sharing).
        If you are happy with the results, press `Transform Files` button at the bottom of the screen. We will not start generation
        until you press the button below.
        """
        )


    @pn.depends('file_input.value', watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        fname = self.file_input.filename

        if value:
            self.static_text.value = check_file(fname, value)

        else:
            print("error")

#     @pn.depends('txt', watch=True)
#     def _parse_txt(self):

    def view(self):
        return pn.Column(
            self.markdown,
            "## Upload data",
            self.file_input,
            self.static_text,
            self.gen_btn
        )

In [8]:
import param


class FileSelector(param.Parameterized):
    gen_btn = pn.widgets.Button(name='Transform Files', button_type='primary')
    file_in = param.Parameter()
    txt     = param.String()


    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.FileInput(), **params)
        self.static_text = pn.widgets.StaticText()

        self.markdown = pn.pane.Markdown("""
        ## Welcome!

        Here you can select the folder(s) that you want to transform. Beware that we are using recursive walk to select all the content
        available in the specified folder. That means if the folder contains a sub-folder, we will transform the content if it falls
        into the accepted file formats.

        Below you will see the generated folder with content as specified at [BIDS Computational Model Specification](https://docs.google.com/document/d/1NT1ERdL41oz3NibIFRyVQ2iR8xH-dKY-lRCB4eyVeRo/edit?usp=sharing).
        If you are happy with the results, press `Transform Files` button at the bottom of the screen. We will not start generation
        until you press the button below.
        """
        )


    @pn.depends('file_input.value', watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        fname = self.file_input.filename

        if value:
            self.txt = check_file(fname, value)

        else:
            print("error")

    @pn.depends('txt', watch=True)
    def parse_txt(self):
        self.static_text.value = self.txt


    def view(self):
        return pn.Column(
            self.markdown,
            "## Upload data",
            self.file_input,
            self.static_text,
            self.gen_btn
        )

In [9]:
upload = FileSelector()
upload_app_view = upload.view()
# pn.serve(upload_app_view)

In [10]:
# user guide v2
intro = pn.widgets.Button(name='Introduction', button_type='light', value=True, width=100)
sup_files = pn.widgets.Button(name='Supported Files', button_type='light', width=100)
transform = pn.widgets.Button(name='Transforming Data', button_type='light', width=100)
how_to = pn.widgets.Button(name='How to use the app', button_type='light', width=100)
bep034 = pn.widgets.Button(name='BIDS Computational Data Standard', button_type='light', width=100)

ug_sidebar = pn.Row(
    intro,
    sup_files,
    transform,
    how_to,
    bep034,
    margin=(20, 0, 0, 0)
)

ug_sidebar

Row(margin=(20, 0, 0, 0), sizing_mode='stretch_width')
    [0] Button(button_type='light', name='Introduction', sizing_mode='stretch_width', width=100)
    [1] Button(button_type='light', name='Supported Files', sizing_mode='stretch_width', width=100)
    [2] Button(button_type='light', name='Transforming Data', sizing_mode='stretch_width', width=100)
    [3] Button(button_type='light', name='How to use the app', sizing_mode='stretch_width', width=100)
    [4] Button(button_type='light', name='BIDS Computational D..., sizing_mode='stretch_width', width=100)

In [11]:
ug_main = pn.Column(pn.pane.Markdown('''
#### Introduction <a name="intro"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Et ultrices neque ornare aenean euismod elementum nisi. Tincidunt ornare massa eget egestas purus viverra. Quis blandit turpis cursus in hac. Dictum varius duis at consectetur lorem donec. Phasellus faucibus scelerisque eleifend donec pretium vulputate sapien nec. Ipsum nunc aliquet bibendum enim facilisis gravida neque. Odio aenean sed adipiscing diam donec. Massa tincidunt nunc pulvinar sapien et. Nibh nisl condimentum id venenatis a condimentum. Congue quisque egestas diam in arcu cursus. Aenean vel elit scelerisque mauris. Sit amet justo donec enim. Habitant morbi tristique senectus et.


Praesent elementum facilisis leo vel fringilla est ullamcorper. Urna molestie at elementum eu facilisis sed. Maecenas pharetra convallis posuere morbi leo urna molestie. Ultricies lacus sed turpis tincidunt id aliquet risus feugiat in. Facilisi cras fermentum odio eu feugiat pretium nibh ipsum. Sem integer vitae justo eget magna fermentum iaculis. Sollicitudin tempor id eu nisl nunc mi ipsum faucibus vitae. Mauris augue neque gravida in fermentum. Scelerisque eu ultrices vitae auctor eu augue. Sed cras ornare arcu dui.


In mollis nunc sed id semper risus. Velit ut tortor pretium viverra suspendisse. Viverra adipiscing at in tellus integer. Ultricies lacus sed turpis tincidunt. Vitae purus faucibus ornare suspendisse. Arcu cursus vitae congue mauris rhoncus aenean vel elit. Vestibulum lorem sed risus ultricies tristique. A erat nam at lectus urna duis convallis. Etiam erat velit scelerisque in dictum non. Sit amet facilisis magna etiam tempor. Purus gravida quis blandit turpis cursus in hac. Ultricies tristique nulla aliquet enim tortor at auctor.


Quam vulputate dignissim suspendisse in est. Adipiscing tristique risus nec feugiat. Faucibus interdum posuere lorem ipsum dolor sit amet consectetur. Et leo duis ut diam quam nulla porttitor massa. Nisl nisi scelerisque eu ultrices vitae. Sit amet cursus sit amet dictum. Sodales ut eu sem integer vitae justo eget magna. Arcu vitae elementum curabitur vitae. Lobortis elementum nibh tellus molestie. Sit amet est placerat in egestas erat imperdiet sed euismod. Quis enim lobortis scelerisque fermentum dui faucibus in. Nunc mattis enim ut tellus. Accumsan tortor posuere ac ut consequat semper viverra nam libero.


Est velit egestas dui id ornare arcu. A arcu cursus vitae congue mauris. Semper feugiat nibh sed pulvinar proin gravida hendrerit lectus. Malesuada fames ac turpis egestas sed tempus urna et pharetra. Tortor posuere ac ut consequat semper. Etiam dignissim diam quis enim. Leo vel orci porta non pulvinar neque laoreet suspendisse interdum. Ligula ullamcorper malesuada proin libero nunc consequat interdum varius. Eu scelerisque felis imperdiet proin fermentum leo vel. Auctor elit sed vulputate mi sit amet mauris. Et netus et malesuada fames ac turpis egestas. Nisl vel pretium lectus quam id. Amet risus nullam eget felis eget nunc. Tincidunt dui ut ornare lectus sit amet est. Eu consequat ac felis donec et odio pellentesque. Aliquam nulla facilisi cras fermentum odio eu feugiat pretium nibh. Et netus et malesuada fames ac turpis. Mi proin sed libero enim sed. Et malesuada fames ac turpis egestas integer eget aliquet.'''))

ug_main

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')

In [12]:
ug_combined = pn.Column(ug_sidebar, ug_main)
ug_combined

Column(sizing_mode='stretch_width')
    [0] Row(margin=(20, 0, 0, 0), sizing_mode='stretch_width')
        [0] Button(button_type='light', name='Introduction', sizing_mode='stretch_width', width=100)
        [1] Button(button_type='light', name='Supported Files', sizing_mode='stretch_width', width=100)
        [2] Button(button_type='light', name='Transforming Data', sizing_mode='stretch_width', width=100)
        [3] Button(button_type='light', name='How to use the app', sizing_mode='stretch_width', width=100)
        [4] Button(button_type='light', name='BIDS Computational D..., sizing_mode='stretch_width', width=100)
    [1] Column(sizing_mode='stretch_width')
        [0] Markdown(str, sizing_mode='stretch_width')

In [13]:
main = pn.Tabs(
    ('Get Started',   FileSelector().view()),
    ('View Contents', FileSelector().view()),
    ('User Guide',    ug_combined)
)

main

Tabs(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_width')
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Markdown(str, sizing_mode='stretch_width')
        [2] FileInput(sizing_mode='stretch_width')
        [3] StaticText(sizing_mode='stretch_width')
        [4] Button(button_type='primary', name='Transform Files', sizing_mode='stretch_width')
    [1] Column(sizing_mode='stretch_width')
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Markdown(str, sizing_mode='stretch_width')
        [2] FileInput(sizing_mode='stretch_width')
        [3] StaticText(sizing_mode='stretch_width')
        [4] Button(button_type='primary', name='Transform Files', sizing_mode='stretch_width')
    [2] Column(sizing_mode='stretch_width')
        [0] Row(margin=(20, 0, 0, 0), sizing_mode='stretch_width')
            [0] Button(button_type='light', name='Introduction', sizing_mode='stretch_width', width=100)
            [1] Button(button_type='light', name='Supported Files', sizing_mode='stretch_width', width=100)
            [2] Button(button_type='light', name='Transforming Data', sizing_mode='stretch_width', width=100)
            [3] Button(button_type='light', name='How to use the app', sizing_mode='stretch_width', width=100)
            [4] Button(button_type='light', name='BIDS Computational D..., sizing_mode='stretch_width', width=100)
        [1] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')

In [14]:
panel = pn.template.FastListTemplate(
    title='Visualize | Transform | Download',
    site='INCF',
    main=main
)

pn.serve(panel)

Launching server at http://localhost:54997


In [3]:
import sys
sys.path.append("..")
from incf.preprocess import preprocess as prep

def check_file(fname=None, value=None):
    s_id = prep.create_uuid()

    # sub-AA_desc-50healthy_weights.tsv
    if fname == 'weights.txt':
        val = create_layout({'name': s_id,
                       'desc': 'default',
                       'fname': fname
                      })
        create_output_folder(sub=f'sub_{s_id}')

        fname = f'sub-{s_id}_desc-default_{os.path.basename(fname)}.tsv'
        string_io = io.StringIO(value.decode("utf8"))
        pd.read_csv(string_io, sep='\t').to_csv(os.path.join('../output', f'sub_{s_id}', 'net', fname), sep='\t',
                                                header=None, index=None)
        return val


def create_output_folder(path='../output', sub='sub_00'):
    # verify folders exist
    check_folders(path)

    # patient specific folders
    sub = os.path.join(path, sub)
    net = os.path.join(sub, 'net')

    if not os.path.exists(sub):
        print(f'Creating folder `{sub}`...')
        os.mkdir(sub)

        print(f'Creating folder `{net}`...')
        os.mkdir(net)

    else:
        # TODO: add create new id creation
        pass


def check_folders(path):
    eq    = os.path.join(path, 'eq')
    code  = os.path.join(path, 'code')
    coor  = os.path.join(path, 'coord')
    param = os.path.join(path, 'param')

    for p in [path, eq, code, coor, param]:
        if not os.path.exists(p):
            print(f'Creating folder `{os.path.basename(p)}`...')
            os.mkdir(p)

    read  = os.path.join(path, 'README.txt')
    part  = os.path.join(path, 'participants.tsv')
    desc  = os.path.join(path, 'dataset_description.json')
    chgs  = os.path.join(path, 'CHANGES.txt')

    for p in [read, part, desc, chgs]:
        if not os.path.exists(p):
            print(f'Creating file `{os.path.basename(p)}`...')
            Path(p).touch()


# create_output_folder()
def create_layout(subs=None):
    return f"""
    output/ <br>
        &emsp;|___ code <br>
        &emsp;|___ coord <br>
        &emsp;|___ eq <br>
        &emsp;|___ param <br>
        &emsp;{create_sub(subs)}
        &emsp;|___ README <br>
        &emsp;|___ CHANGES <br>
        &emsp;|___ dataset_description.json <br>
        &emsp;|___ participants.tsv
    """

def create_sub(sub):
    if sub is not None:
        if sub['fname'] == 'weights.txt':
            return f"""
                |___ sub-{sub['name']} <br>
                     &emsp;&emsp;&emsp;|__ net <br>
                         &emsp;&emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_weights.tsv <br>
                         &emsp;&emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_weights.json <br>
                     &emsp;&emsp;&emsp;|__ spatial <br>
                     &emsp;&emsp;&emsp;|__ ts  <br>
            """


        elif sub['fname'] == 'distances.txt':
            return f"""
                &emsp;|___ sub-{sub['name']} <br>
                     &emsp;&emsp;|__ net <br>
                         &emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_distances.tsv <br>
                         &emsp;&emsp;&emsp;|__ sub-{sub['name']}_desc-{sub['desc']}_distances.json <br>
                     &emsp;&emsp;|__ spatial <br>
                     &emsp;&emsp;|__ ts  <br>
            """
    return ''
#         return f"""
#             |___ sub-{sub['name']}
#                  |__ net
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_distances.tsv
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_distances.json
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_weights.tsv
#                      |__ sub-{sub['name']}_desc-{sub['desc']}_weights.json
#                  |__ spatial
#                  |__ ts
#         """



In [4]:
import param


class FileSelector(param.Parameterized):
    gen_btn = pn.widgets.Button(name='Transform Files', button_type='primary')
    file_in = param.Parameter()


    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.FileInput(), **params)
        self.static_text = pn.widgets.StaticText()

        self.markdown = pn.pane.Markdown("""
        ## Welcome!

        Here you can select the folder(s) that you want to transform. Beware that we are using recursive walk to select all the content
        available in the specified folder. That means if the folder contains a sub-folder, we will transform the content if it falls
        into the accepted file formats.

        Below you will see the generated folder with content as specified at [BIDS Computational Model Specification](https://docs.google.com/document/d/1NT1ERdL41oz3NibIFRyVQ2iR8xH-dKY-lRCB4eyVeRo/edit?usp=sharing).
        If you are happy with the results, press `Transform Files` button at the bottom of the screen. We will not start generation
        until you press the button below.
        """
        )


    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        value = self.file_input.value
        fname = self.file_input.filename

        if value:
            self.static_text.value = check_file(fname, value)
        else:
            print("error")

    def view(self):
        return pn.Column(
            self.markdown,
            "## Upload data",
            self.file_input,
            self.static_text,
            self.gen_btn
        )

In [5]:
upload = FileSelector()
upload_app_view = upload.view()
pn.serve(upload_app_view)

Launching server at http://localhost:58888


Creating folder `../output\sub_ww69l`...
Creating folder `../output\sub_ww69l\net`...


In [ ]:
# descriptor for JSONEditor usage
markdown = pn.pane.Markdown("""
## Welcome!

Here you can select the folder(s) that you want to transform. Beware that we are using recursive walk to select all the content
available in the specified folder. That means if the folder contains a sub-folder, we will transform the content if it falls
into the accepted file formats.

Below you will see the generated folder with content as specified at [BIDS Computational Model Specification](https://docs.google.com/document/d/1NT1ERdL41oz3NibIFRyVQ2iR8xH-dKY-lRCB4eyVeRo/edit?usp=sharing).
If you are happy with the results, press `Transform Files` button at the bottom of the screen. We will not start generation
until you press the button below.

### Select Folder(s)
"""
)


generated_md = pn.pane.Markdown("""
### Generated Folder

```
tsv/
    example1.tsv
    example2.tsv
json/
    example1.json
    example2.json

```
""")

# get current path
path = os.getcwd()
change_path = pn.widgets.FileSelector(path)

generate_btn = pn.widgets.Button(name='Transform Files', button_type='primary')
generate_btn.on_click(gen_watch)

main_area = pn.Column(
                markdown,
                change_path,
                generated_md,
                generate_btn
            )
main_area

In [39]:
import param

GET_STARTED = """
## Welcome!
Here you can select the folder(s) that you want to transform. Beware that we are using recursive walk to select all the content available in the specified folder. That means if the folder contains a sub-folder, we will transform the content if it falls into the accepted file formats.
Below you will see the generated folder with content as specified at [BIDS Computational Model Specification](https://docs.google.com/document/d/1NT1ERdL41oz3NibIFRyVQ2iR8xH-dKY-lRCB4eyVeRo/edit?usp=sharing). If you are happy with the results, press `Transform Files` button at the bottom of the screen. We will not start generation until you press the button below.
"""


class MainArea(param.Parameterized):
    # generate files button
    gen_btn = pn.widgets.Button(name='Generate files', button_type='primary')

    def __init__(self, **params):
        super().__init__(text_input=pn.widgets.TextInput(name='Insert Path'),
                         file_selector= pn.widgets.FileSelector(directory=os.getcwd(), root_directory='~'),
                         **params)
#         self.file_selector = pn.widgets.FileSelector(directory=os.getcwd(), root_directory='~',
#                                                      only_files=False, refresh_period=10)

    @pn.depends('text_input.value', watch=True)
    def _select_path(self):
        if os.path.exists(self.text_input.value):
            self.file_selector.directory = self.text_input.value

            self.file_selector._selector.options.clear()
            self.file_selector._selector.options.values = create_ordered_dict(os.listdir(self.text_input.value))

            print(dir(self.file_selector._selector.options))
            print(self.file_selector._selector.options.values)


#             print(self.file_selector._selector.options)
#             print(create_ordered_dict(os.listdir(self.text_input.value)))


    def view(self):
        return pn.Tabs(
                ('Select Files', pn.Column(pn.pane.Markdown(GET_STARTED),
                                           self.text_input,
                                           self.file_selector,
                                           self.gen_btn)))

ma = MainArea()
pn.serve(ma.view())

Launching server at http://localhost:64163


['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'move_to_end', 'pop', 'popitem', 'setdefault', 'update', 'values']
OrderedDict([('app_templates', 'C:\\Users\\dinar\\Desktop\\gsoc\\app_templates\\app_templates'), ('AUTHORS.rst', 'C:\\Users\\dinar\\Desktop\\gsoc\\app_templates\\AUTHORS.rst'), ('CONTRIBUTING.rst', 'C:\\Users\\dinar\\Desktop\\gsoc\\app_templates\\CONTRIBUTING.rst'), ('data', 'C:\\Users\\dinar\\Desktop\\gsoc\\app_templates\\data'), ('docs', 'C:\\Users\\dinar\\Desktop\\gsoc\\app_templates\\docs'), ('examples',

In [19]:
# def create_ordered_dict(vals):

from collections import OrderedDict

od = OrderedDict()
od['components.ipynb'] = 'C:\\Users\\dinar\\Desktop\\gsoc\\app_templates\\components.ipynb'

od

OrderedDict([('components.ipynb',
              'C:\\Users\\dinar\\Desktop\\gsoc\\app_templates\\components.ipynb')])

In [30]:
def create_ordered_dict(vals):
    od = OrderedDict()

    for val in vals:
        if val.startswith('.'): continue

        od[os.path.basename(val)] = os.path.abspath(val)

    return od

In [20]:
os.path.basename('C:\\Users\\dinar\\Desktop\\gsoc\\app_templates\\components.ipynb')

'components.ipynb'